<a href="https://colab.research.google.com/github/arampacha/fastai_nbs/blob/main/lm_finetuning_fastai_blurr_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Mar 25 21:26:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq --no-deps fastai ohmeow-blurr
    !pip install -Uqq transformers datasets wandb 
    !pip install -Uqq fastcore onnx onnxruntime sentencepiece seqeval rouge-score

In [5]:
from fastai.text.all import *
from fastai.callback.wandb import *

In [6]:
def read_text(fn):
    return open(fn).read()

In [7]:
path = untar_data(URLs.IMDB)

## Setup

In [8]:
model_name = 'roberta-base'

max_len = 512
bs = 8
val_bs = 16

## Tracking

In [9]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
import wandb

WANDB_NAME = f'imdb-{model_name}-simple'
GROUP = 'IMDB'
NOTES = f'Simple finetuning {model_name} with RAdam lr=2e-5'
CONFIG = {}
TAGS =[model_name,'imdb','radam','simple']

In [11]:
wandb.init(reinit=True, project="lm-finetuning", entity="arampacha", 
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG)

wandb: Currently logged in as: fastai_community (use `wandb login --relogin` to force relogin)


## Training

In [14]:
from transformers import *

from blurr.data.all import *
from blurr.modeling.all import *

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [15]:
task = HF_TASKS_AUTO.SequenceClassification
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(model_name, task=task, tokenizer_kwargs={'max_len':512})

In [16]:
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), CategoryBlock)
dblock = DataBlock(blocks=blocks, 
                   get_items=get_text_files,
                   get_x = read_text,
                   get_y=parent_label,
                   splitter=GrandparentSplitter(valid_name='test'))

dls = dblock.dataloaders(path, bs=bs, val_bs=val_bs, shuffle=True)

In [17]:
model = HF_BaseModelWrapper(hf_model)
learn = Learner(dls,
                model,
                opt_func=RAdam,
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.create_opt() 
learn.unfreeze()
learn.blurr_summary()

HF_BaseModelWrapper (Input shape: 8 x 512)
Layer (type)         Output Shape         Param #    Trainable 
                     8 x 512 x 768       
Embedding                                 38603520   True      
Embedding                                 394752     True      
Embedding                                 768        True      
LayerNorm                                 1536       True      
Dropout                                                        
Linear                                    590592     True      
Linear                                    590592     True      
Linear                                    590592     True      
Dropout                                                        
Linear                                    590592     True      
LayerNorm                                 1536       True      
Dropout                                                        
____________________________________________________________________________
      

### simple finetuning

In [18]:
learn.fit_one_cycle(4, 2e-5, cbs=WandbCallback(log_preds=False, log_model=False))

Could not gather input dimensions


epoch,train_loss,valid_loss,accuracy,time
0,0.210271,0.150023,0.945360,16:37
1,0.152272,0.193026,0.940520,16:41
2,0.051281,0.145123,0.956000,16:45
3,0.014714,0.179491,0.955920,16:40


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
